In [2]:
import pandas as pd

# API Calls
import requests

# XML Processing
import xml.etree.ElementTree as ET

In [14]:
def get_game_info(game_id):
    url = f'https://www.boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1'
    response = requests.get(url)
    xml_data = ET.fromstring(response.content)
    return xml_data

game_ids = [1927, 13, 234]

def extract_game_info(xml_data):
    game_info = {}
    
    # Extrahiere Spiel-ID aus dem Wurzelelement
    info = xml_data.find(".//item").attrib['id']

    return info

print(extract_game_info(get_game_info(1927)))

1927


In [16]:
# Function to pretty print the XML tree
def prettify(elem, level=0):
    """Return a pretty-printed XML string for the Element."""
    indent = "    "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = "\n" + indent * (level + 1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = "\n" + indent * level
        for subelem in elem:
            prettify(subelem, level + 1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = "\n" + indent * level
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = "\n" + indent * level

# Pretty print the XML tree
prettify(get_game_info(1927))
print(ET.tostring(get_game_info(1927), encoding='unicode'))

<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="1927">
         <thumbnail>https://cf.geekdo-images.com/J-ts3MW0UhDzs621TR6cog__thumb/img/8hVkpMC5pDLr6ARI_4gI4N3aF5M=/fit-in/200x150/filters:strip_icc()/pic1871016.jpg</thumbnail>
      <image>https://cf.geekdo-images.com/J-ts3MW0UhDzs621TR6cog__original/img/FbqPPCilgZKND2xmhWJgkfjZiYE=/0x0/filters:format(jpeg)/pic1871016.jpg</image>
                                     				
				<name type="primary" sortindex="1" value="Munchkin" />
			
						                               				
				<name type="alternate" sortindex="1" value="Munchkin 1+2" />
			    				
				<name type="alternate" sortindex="1" value="Munchkin Das Kartenspiel" />
			    				
				<name type="alternate" sortindex="1" value="Munchkin Deluxe" />
			    				
				<name type="alternate" sortindex="1" value="Munchkin Edycja 2012" />
			    				
				<name type="alternate" sortindex="1" value="Munchkin Foil Edition" />
			    				
				<nam

In [43]:
def get_game_info(game_id):
    url = f'https://www.boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1'
    response = requests.get(url)
    xml_data = ET.fromstring(response.content)
    return xml_data

def extract_game_info(xml_data):
    game_info = {}
    
    # Extrahiere Spiel-ID aus dem Wurzelelement
    game_info['id'] = xml_data.find(".//item").attrib['id']

    # Extract the game name
    game_info['game_name'] = xml_data.find(".//name[@type='primary']").attrib['value']

    # Board Game Category
    categories = xml_data.findall(".//link[@type='boardgamecategory']")
    game_info['category'] = [category.attrib['value'] for category in categories] if categories else None

    # Board Game Mechanic
    mechanics = xml_data.findall(".//link[@type='boardgamemechanic']")
    game_info['mechanic'] = [mechanic.attrib['value'] for mechanic in mechanics] if mechanics else None

    # Extract the publishing year
    game_info['year'] = xml_data.find(".//yearpublished").attrib['value']
    
    # Extrahiere Designer-Informationen
    designers = xml_data.findall(".//link[@type='boardgamedesigner']")
    game_info['designer'] = [designer.attrib['value'] for designer in designers] if designers else None

    # Extrahiere Spielerinformationen
    min_players = xml_data.find(".//minplayers").attrib['value']
    max_players = xml_data.find(".//maxplayers").attrib['value'] 
    game_info['player_no'] = f"{min_players}-{max_players}"

    return game_info

# Beispiel-Spiel-IDs
game_ids = [1927, 13, 234]

# Daten für jeden Spiel-ID abrufen und in einen DataFrame umwandeln
game_data = [extract_game_info(get_game_info(game_id)) for game_id in game_ids]
df = pd.DataFrame(game_data)

# DataFrame anzeigen
print(df)




     id                    game_name                               category  \
0  1927                     Munchkin  [Card Game, Fantasy, Fighting, Humor]   
1    13                        CATAN                [Economic, Negotiation]   
2   234  Hannibal: Rome vs. Carthage          [Ancient, Political, Wargame]   

                                            mechanic  year  \
0  [Hand Management, Take That, Variable Player P...  2001   
1  [Chaining, Dice Rolling, Hexagon Grid, Income,...  1995   
2  [Area Majority / Influence, Campaign / Battle ...  1996   

              designer player_no  
0  [Steve Jackson (I)]       3-6  
1       [Klaus Teuber]       3-4  
2     [Mark Simonitch]       2-2  
